This is to calculate the average framewise displacement for participants based on the confounds file generated from fmriprep

Steps:
- create functions to grab the framewise_displacement row from each file
- then compute the average of that row (excluding NA)
- write this to new CSV

In [1]:
#import modules
import os
import re
import csv, operator
import pandas as pd

In [12]:
#define average computation function
def computeAvg(file, fdmap):
    with open(file, 'r') as infile:
        #reader = csv.DictReader(infile, delimiter='\t', skipinitialspace=True)
        reader = pd.read_table(infile)
        reader['framewise_displacement'].fillna(0) #to replace NA with zero
        count = 0
        sum = 0.0
        for row in reader:
            val = row['framewise_displacement']
            try:
                f = float(val) #this is to discard NAs and not count them
                sum += f
                count += 1
            except:
                continue

        fdmap[file] = sum / count if count  != 0 else 0

In [13]:
#define file names (confouds tsv file to read)
def calcavg(sdir, fdmap):
    os.chdir(sdir)
    files = os.listdir()
    for file in files:
        if re.search("task-task_desc-confounds_regressors.tsv", file):
            computeAvg(file, fdmap)

In [14]:
#to get directories
def explore(dir, fdmap):
    os.chdir(dir)
    subdirs = os.listdir()
    for sdir in subdirs:
        if os.path.isdir(sdir) and sdir == "func":
            cwd = os.getcwd()
            calcavg(sdir, fdmap)
            os.chdir(cwd)

In [17]:
os.getcwd()

'/imaging/su01/RED/bids/derivatives/fmriprep-1.5.0/fmriprep/sub-053/func'

In [15]:
#run code
if __name__ == '__main__':

    # you can set the os.environ['OUTPUT_FILE'] and os.environ['INPUT_PATH'] below or,
    # outside from this app
    os.environ['OUTPUT_FILE'] = os.getcwd() + '/' + 'result.tsv'
    os.environ['INPUT_PATH']  = os.getcwd()
    
    fptr = open(os.environ['OUTPUT_FILE'], 'w')
    framewise_displacements = dict() # initialize map
    subdirs = os.listdir()
    for sdir in subdirs:
        if os.path.isdir(sdir):
            cwd = os.getcwd()
            explore(sdir,framewise_displacements)
            os.chdir(cwd)
            
    #sort the file alphabetically by filename
    fw_sorted_displacements = sorted(framewise_displacements.items())
    
    #write to csv file
    fw_disp_dicts = [{'file': f, 'framewise displacement average': avg} for f,avg in fw_sorted_displacements]
    names = ['file', 'framewise displacement average']
    with open('RED_test_fdavg.csv', mode='wt') as avg_file:
        csv_writer = csv.DictWriter(avg_file, delimiter=',', fieldnames=names, lineterminator='\n')
        csv_writer.writeheader()
        for row in fw_disp_dicts:
            csv_writer.writerow(row)


In [34]:
## Don't need to run this but this is another option using numpy
## Not as efficient as one above but we can adjust what we sort alphabetically by here
## in operator.itemgetter(0) <- using indexing

#sort rows alphabetically by file name just for ease of reading
data = csv.reader(open('[file_from_above].csv'), delimiter=';')
sortedlist = sorted(data, key=operator.itemgetter(0), reverse=False)

import numpy as np
np.savetxt("[newfile].csv",
            sortedlist,
            delimiter=", ",
            fmt='% s')